In [2]:
from pyspark.sql import functions as F
from pyspark.sql.functions import isnan, when, count, col
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
sc.install_pypi_package("pandas==0.25.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached pandas-0.25.1-cp36-cp36m-manylinux1_x86_64.whl (10.5 MB)
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)

In [4]:
import pandas as pd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Github Commands 

In [5]:
# !github ..... commands not working

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Sample Data

In [ ]:
input_bucket = 's3://amazon-reviews-pds'
input_path = '/parquet/product_category=Books/*.parquet'
df = spark.read.parquet(input_bucket + input_path)

In [9]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|year|
+-----------+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+----+
|         US|   15444933|R1WWG70WK9VUCH|1848192576|     835940987|Standing Qigong f...|          5|            9|         10|   N|                Y|Informative AND i...|After attending a...| 2015-05-02|2015|
|         US|   20595117|R1EQ3POS0RIOD5|145162445X|     574044348|A Universe from N...|          4|            4|          7|   N|                N|Between 'Nothing'...

In [3]:
df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(marketplace='US', customer_id='15444933', review_id='R1WWG70WK9VUCH', product_id='1848192576', product_parent='835940987', product_title='Standing Qigong for Health and Martial Arts - Zhan Zhuang', star_rating=5, helpful_votes=9, total_votes=10, vine='N', verified_purchase='Y', review_headline='Informative AND interesting!', review_body="After attending a few Qigong classes, I wanted to have a book to read and re-read the instructions so I could practice at home.  I also wanted to gain more of an understanding of the purpose and benefit of the movements in order to practice them with a more focused purpose.<br /><br />The book exceeded my expectations.  The explanations are very clear and are paired with photos showing the correct form.  The book itself is more than just the Qigong, it's a very interesting read.  I read the whole book in two days and will read it again. I rarely read books twice!  The book has provided the information and additional instruction that I was looking f

In [4]:
df.describe()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string, review_headline: string, review_body: string, year: string]

In [6]:
df.count(
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

20726160

# Import Real Data from S3  

In [44]:
path =  's3://bosch-data-spark/data/train_numeric.csv'
df_train_numeric = spark.read.load(path,format="csv",inferSchema = True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
len(df_train_numeric.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

970

In [46]:
df_train_numeric.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1183747

In [47]:
df_train_numeric.select(df_train_numeric[-1]).distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2

# Not using Categorical Features for Now  

In [10]:
path =  's3://bosch-data-spark/data/train_categorical.csv'
df_train_categorical = spark.read.load(path,format="csv",header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_train_categorical.limit(100).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

     Id L0_S1_F25 L0_S1_F27  ... L3_S49_F4237 L3_S49_F4239 L3_S49_F4240
0     4      None      None  ...         None         None         None
1     6      None      None  ...         None         None         None
2     7      None      None  ...         None         None         None
3     9      None      None  ...         None         None         None
4    11      None      None  ...         None         None         None
..  ...       ...       ...  ...          ...          ...          ...
95  192      None      None  ...         None         None         None
96  197      None      None  ...         None         None         None
97  200      None      None  ...         None         None         None
98  201      None      None  ...         None         None         None
99  205      None      None  ...         None         None         None

[100 rows x 2141 columns]

# Start of numerical feature exploration

In [12]:
df_train_numeric.select(df_train_numeric[0]).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+
| Id|
+---+
|  4|
|  6|
|  7|
|  9|
| 11|
+---+
only showing top 5 rows

In [13]:
df_train_categorical.select(df_train_categorical[0]).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+
| Id|
+---+
|  4|
|  6|
|  7|
|  9|
| 11|
+---+
only showing top 5 rows

# NOTE: On Join for PySpark, indicate the column match as a string rather than the commented out statement 

In [14]:
# inner_join = df_train_numeric.join(df_train_categorical, df_train_numeric.Id == df_train_categorical.Id )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# On Exploring the Categorical Dataset, it appears there are so many NaN's that it simply doesn't make sense to analyze it

In [15]:
# inner_join = df_train_numeric.join(df_train_categorical, "Id" )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# len(inner_join.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# count all nan values for all columns
# inner_join.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in inner_join.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
df_train_numeric.head(1)[0][:100]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(4, 0.03, -0.034, -0.197, -0.179, 0.118, 0.116, -0.015, -0.032, 0.02, 0.083, -0.273, -0.273, -0.271, 0.167, -0.213, -0.023, -0.192, -0.088, 0.001, 0, 0.01, -0.223, -0.03, None, None, None, None, None, None, None, None, None, -0.001, -0.004, None, None, None, None, None, -0.164, -0.077, 0.06, -0.157, 0.0, 0.001, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 0.018, -0.136, -0.013, 0.106, -0.112, -0.131, -0.021, -0.097, 0.26, 0.012, -0.115, 0.113, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)

In [33]:
def df_wrangle(df):
    df =  df.drop("Id","Response")
    df = df.select([col(c).cast("double") for c in df.columns])
    imputer = Imputer(
    inputCols=df.columns, 
    outputCols=["{}_imputed".format(c) for c in df.columns]
    )
    df = imputer.fit(df).transform(df)
    df = df[imputer.getOutputCols()]
    return df 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
def df_preprocess(df):
    assembler = VectorAssembler(
    inputCols = df.columns,
    outputCol = "features"
    )
    df = assembler.transform(df)
#     df.select("features").show(truncate=False)
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
df_train_numeric_imputed = df_wrangle(df_train_numeric)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
df_train_numeric_imputed.head(1)[0][:100]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(0.03, -0.034, -0.197, -0.179, 0.118, 0.116, -0.015, -0.032, 0.02, 0.083, -0.273, -0.273, -0.271, 0.167, -0.213, -0.023, -0.192, -0.088, 0.001, 0.0, 0.01, -0.223, -0.03, -0.0003108022515146497, -8.989925547050625e-06, 0.00043946365189956327, -0.00010633447661842699, -3.069839979923448e-05, -5.9753576251538656e-05, -2.2550999677299486e-05, -0.0001611494197927842, 0.0001058594356872173, -0.001, -0.004, 6.560298310517508e-05, 0.00017431195362756688, -0.00027867653132211657, -0.00016086705134108002, 0.00013200467273175753, -0.164, -0.077, 0.06, -0.157, 0.0, 0.001, 0.00011895825116658523, 0.000157033472469625, 1.9940532899666376e-05, 3.442072333147556e-05, 0.00019083661631044175, 2.3974226598538684e-05, -4.2723755755363216e-05, 5.233560372065071e-06, -0.00015426373421844163, -2.659765398233033e-05, -1.858113348016249e-05, 1.835512875622714e-05, -3.132863893676562e-05, 6.329384519462659e-05, -1.8184895357793733e-05, -5.411062616903939e-06, -7.771141486615497e-05, -2.6647604031658774e-05, -1.

In [54]:
len(df_train_numeric_imputed.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

968

In [61]:
df_preprocessed = df_preprocess(df_train_numeric_imputed)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Vectorization Was A Success 

In [89]:
df_preprocessed.limit(1).toPandas()["features"][0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DenseVector([0.03, -0.034, -0.197, -0.179, 0.118, 0.116, -0.015, -0.032, 0.02, 0.083, -0.273, -0.273, -0.271, 0.167, -0.213, -0.023, -0.192, -0.088, 0.001, 0.0, 0.01, -0.223, -0.03, -0.0003, -0.0, 0.0004, -0.0001, -0.0, -0.0001, -0.0, -0.0002, 0.0001, -0.001, -0.004, 0.0001, 0.0002, -0.0003, -0.0002, 0.0001, -0.164, -0.077, 0.06, -0.157, 0.0, 0.001, 0.0001, 0.0002, 0.0, 0.0, 0.0002, 0.0, -0.0, 0.0, -0.0002, -0.0, -0.0, 0.0, -0.0, 0.0001, -0.0, -0.0, -0.0001, -0.0, -0.0, -0.0, -0.0001, -0.0, -0.0, 0.0, 0.018, -0.136, -0.013, 0.106, -0.112, -0.131, -0.021, -0.097, 0.26, 0.012, -0.115, 0.113, -0.0001, -0.0002, -0.0001, -0.0003, 0.0001, 0.0002, 0.0, -0.0001, -0.0002, 0.0002, 0.0001, 0.0001, -0.0, 0.0, 0.0005, 0.0005, -0.0002, -0.0003, 0.0, -0.0, 0.0001, 0.0004, 0.0002, -0.0003, -0.0001, -0.0004, -0.0004, 0.0, -0.0, -0.0003, 0.0003, -0.0001, -0.0003, 0.0001, -0.0, 0.0002, -0.0002, -0.0004, -0.0002, -0.0003, -0.0005, -0.0, -0.0003, 0.0, -0.0003, -0.0002, -0.0001, -0.0002, 0.0002, 0.0001, 0.0

# Ready For Machine Learning  

In [112]:
df_train_numeric.select(df_train_numeric.columns[:10]).limit(5).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   L0_S0_F0_imputed  L0_S0_F2_imputed  ...  L0_S0_F16_imputed  L0_S0_F18_imputed
0          0.030000         -0.034000  ...       2.000000e-02           0.083000
1          0.000055          0.000091  ...       6.722445e-07           0.000005
2          0.088000          0.086000  ...      -2.250000e-01          -0.147000
3         -0.036000         -0.064000  ...      -2.600000e-02          -0.046000
4         -0.055000         -0.086000  ...      -1.690000e-01          -0.099000

[5 rows x 10 columns]

In [ ]:
spark.sql("SELECT C* FROM table1 where ")